In [1]:
##this is to test if results of model are good ennough


from nltk.stem.porter import *
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer
st = LancasterStemmer()

import pandas as pd

#import gensim
from gensim import corpora, models, similarities
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

dirc = r'C:\\Users\\DD\\OneDrive\\Documents\\Rapid_Assessment_Tools\\Plans_test\\'

C:\Users\DD\Anaconda2\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [100]:
#load file to list 'paragraphs' split by paragraph

f = open(dirc + 'Bhutan_1_utf8.txt', 'r').read().split("\n\n")
f = [i.replace('\n', '') for i in f]

paragraphs = []
for lines in f:
    if len(lines) > 120:
        paragraphs.append(lines)
len(paragraphs)

1331

In [101]:
#tokenization
p_tokenized = [[word.lower() for word in word_tokenize(document.decode('utf-8'))] for document in paragraphs]
len(p_tokenized)

1331

In [102]:
#remove stopwords
english_stopwords = stopwords.words('english')
p_no_stopwords = [[word for word in document if not word in english_stopwords] for document in p_tokenized]
print p_no_stopwords[1220]

[u'targeted', u'poverty', u'reduction', u'interventions', u',', u'four', u'villages', u'namely', u'dangochang', u',', u'chebesa', u',', u'barchong', u'tshochekha', u'supported', u'reap', u'phase', u'ii', u'.', u'income', u'generating', u'sustainable', u'livelihood', u'activities', u',', u'capacity', u'development', u'establishment', u'self-help', u'groups/cooperatives', u'interventions', u'village', u'level', u'additional', u'support', u'household', u'levels', u'provided', u'poorest', u'households', u'within', u'village', u'.', u'funds', u'programme', u'directly', u'provided', u'centre', u'.']


In [103]:
#remove punctuations
english_punctuations = [',', '.', ':', ';', '?', '(', ')', '[', ']', '&', '!', '*', '@', '#', '$', '%', '...'] #add '...'
p_filtered = [[word for word in document if not word in english_punctuations] for document in p_no_stopwords]
print p_filtered[1220]

[u'targeted', u'poverty', u'reduction', u'interventions', u'four', u'villages', u'namely', u'dangochang', u'chebesa', u'barchong', u'tshochekha', u'supported', u'reap', u'phase', u'ii', u'income', u'generating', u'sustainable', u'livelihood', u'activities', u'capacity', u'development', u'establishment', u'self-help', u'groups/cooperatives', u'interventions', u'village', u'level', u'additional', u'support', u'household', u'levels', u'provided', u'poorest', u'households', u'within', u'village', u'funds', u'programme', u'directly', u'provided', u'centre']


In [104]:
#stemming
p_stemmed = [[st.stem(word) for word in docment] for docment in p_filtered]
print p_stemmed[1220]

[u'target', u'poverty', u'reduc', u'interv', u'four', u'vil', u'nam', u'dangochang', u'chebes', u'barchong', u'tshochekha', u'support', u'reap', u'phas', u'ii', u'incom', u'gen', u'sustain', u'liv', u'act', u'capac', u'develop', u'est', u'self-help', u'groups/cooperative', u'interv', u'vil', u'level', u'addit', u'support', u'household', u'level', u'provid', u'poorest', u'household', u'within', u'vil', u'fund', u'program', u'direct', u'provid', u'cent']


In [105]:
#remove word with low frequency
all_stems = sum(p_stemmed, [])
stems_once = set(stem for stem in set(all_stems) if all_stems.count(stem) == 1)
texts = [[stem for stem in text if stem not in stems_once] for text in p_stemmed]

In [2]:
######## load goals

#use SDG as query

targets = pd.read_excel(dirc +'SDG sub goals.xlsx', names = ['ID', 'Target'],sheetname = 'Sheet1')
targets.head()

,ID,Target
0,1.1,"1.1 By 2030, eradicate extreme poverty for all..."
1,1.2,"1.2 By 2030, reduce at least by half the propo..."
2,1.3,1.3 Implement nationally appropriate social pr...
3,1.4,"1.4 By 2030, ensure that all men and women, in..."
4,1.5,"1.5 By 2030, build the resilience of the poor ..."


In [4]:
#make a list with goal contents

goal = []
for doc in targets.iloc[:]['Target']:
    goal.append(doc)

goal_id = targets.iloc[:]['ID']
goal[10] #test goal 2.4
#goal_id[10]

u'2.4 By 2030, ensure sustainable food production systems and implement resilient agricultural practices that increase productivity and production, that help maintain ecosystems, that strengthen capacity for adaptation to climate change, extreme weather, drought, flooding and other disasters and that progressively improve land and soil quality\xa0'

In [108]:
#tokenization
goal_tokenized = [[word.lower() for word in word_tokenize(document)] for document in goal]

#remove stopwords
goal_no_sw = [[word for word in document if not word in english_stopwords] for document in goal_tokenized]

#stemming
goal_stemmed = [[st.stem(word) for word in l] for l in goal_no_sw]
len(goal_stemmed[10])

38

In [109]:
g_query = goal_stemmed #_frequent
len(g_query[10])

38

In [ ]:
####### match

In [110]:
dictionary = corpora.Dictionary(texts)

2017-02-09 22:33:33,306 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2017-02-09 22:33:33,509 : INFO : built Dictionary(3046 unique tokens: [u'lhuents', u'1200', u'bear', u'four', u'phajod']...) from 1331 documents (total 56199 corpus positions)


In [111]:
corpus = [dictionary.doc2bow(text) for text in texts]
tfidf = models.TfidfModel(corpus)

2017-02-09 22:33:34,825 : INFO : collecting document frequencies
2017-02-09 22:33:34,828 : INFO : PROGRESS: processing document #0
2017-02-09 22:33:34,844 : INFO : calculating IDF weights for 1331 documents and 3045 features (44303 matrix non-zeros)


In [112]:
corpus_tfidf = tfidf[corpus]

In [113]:
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=100)
index = similarities.MatrixSimilarity(lsi[corpus])

2017-02-09 22:33:37,979 : INFO : using serial LSI version on this node
2017-02-09 22:33:37,980 : INFO : updating model with new documents
2017-02-09 22:33:38,174 : INFO : preparing a new chunk of documents
2017-02-09 22:33:38,198 : INFO : using 100 extra samples and 2 power iterations
2017-02-09 22:33:38,200 : INFO : 1st phase: constructing (3046L, 200L) action matrix
2017-02-09 22:33:38,266 : INFO : orthonormalizing (3046L, 200L) action matrix
2017-02-09 22:33:38,571 : INFO : 2nd phase: running dense svd on (200L, 1331L) matrix
2017-02-09 22:33:38,673 : INFO : computing the final decomposition
2017-02-09 22:33:38,677 : INFO : keeping 100 factors (discarding 25.261% of energy spectrum)
2017-02-09 22:33:38,697 : INFO : processed documents up to #1331
2017-02-09 22:33:38,703 : INFO : topic #0(6.418): 0.166*"develop" + 0.145*"dzongkh" + 0.139*"perc" + 0.135*"poverty" + 0.134*"program" + 0.131*"plan" + 0.128*"produc" + 0.115*"serv" + 0.114*"ii" + 0.114*"tour"
2017-02-09 22:33:38,707 : INFO

In [114]:
ml_bhutan = g_query[10]
ml_bow = dictionary.doc2bow(ml_bhutan)
ml_lsi = lsi[ml_bow]
print ml_lsi

[(0, 1.2972206589269133), (1, 0.088925257383483919), (2, 0.24808711278194165), (3, -0.65466234974048498), (4, -0.095503566459803055), (5, 0.47534347972603308), (6, 0.0067641619610999005), (7, -0.05965910476917835), (8, 0.35082223425362685), (9, 0.070181935471937643), (10, 0.63539622162843823), (11, 0.47411219713726344), (12, 0.10613372993319391), (13, 0.36859680340839035), (14, -0.09249730397019873), (15, 0.27517854971093697), (16, 0.56526586870670392), (17, -0.085797960376535337), (18, -0.1723030294707322), (19, 0.28553960638127313), (20, -0.16119975451707441), (21, -0.56237564788452421), (22, -0.15957021979545713), (23, 0.2367713354665707), (24, -0.22981334140254675), (25, -0.072094376353957004), (26, -0.01989236020951788), (27, -0.21599411078623534), (28, 0.25339044298429186), (29, -0.29076714003021964), (30, -0.26774418324492827), (31, 0.25825989764162616), (32, 0.12278677400326508), (33, -0.53004866005415541), (34, -0.29410345720288689), (35, 0.33581396690152621), (36, -0.27163513

In [115]:
sims = index[ml_lsi]
sort_sims = sorted(enumerate(sims), key=lambda item: -item[1])
print sort_sims[0:100]

[(1200, 0.66191697), (1073, 0.64783651), (1247, 0.60438156), (913, 0.59683526), (955, 0.59352064), (961, 0.58773202), (1205, 0.58550191), (546, 0.57644147), (888, 0.56564564), (1100, 0.55881101), (1263, 0.55172247), (1223, 0.55079901), (1258, 0.54877734), (893, 0.5242762), (851, 0.52047467), (548, 0.51088679), (592, 0.50845969), (985, 0.49169022), (963, 0.48994967), (547, 0.4875178), (794, 0.48632097), (873, 0.48593187), (1265, 0.48093945), (981, 0.47951704), (531, 0.47765028), (536, 0.47104299), (146, 0.47097301), (568, 0.46413511), (1242, 0.46098131), (1044, 0.45911366), (1029, 0.45651177), (1081, 0.45338455), (939, 0.44186842), (865, 0.44036543), (847, 0.43858761), (957, 0.43650028), (1142, 0.43411362), (1215, 0.43320528), (796, 0.43110844), (321, 0.4306623), (544, 0.42612472), (1054, 0.4224385), (1079, 0.41344863), (1090, 0.41108328), (760, 0.40901059), (60, 0.40807366), (930, 0.40752453), (1030, 0.40519369), (1147, 0.4043867), (1105, 0.40372801), (277, 0.40062758), (366, 0.3914961

In [119]:
paragraphs[1200] #ranking 1

'Trongsa has suitable climatic conditions which is conducive for enhancing agricultural and horticultural products, besides livestock products such as poultry, piggery and dairy backyard farming. '

In [122]:
paragraphs[147] #ranking 100

'With a strong emphasis on improving the health and education outcomes, achieving progress in the HAI may not be a major constraint other than improving the adult literacy rate which may pose a challenge. However, improving the EVI will require special efforts particularly in addressing economic imbalances, productive capacities and diversification challenges. This will not be easy. Hence, the goodwill, support and cooperation of our development partner in the coming years will be critical for Bhutan. Furthermore, even beyond 2020, Bhutan may require continued cooperation and support from its development partners to ensure that the transition will help establish a solid economic foundation that is dynamic and resilient. '

In [6]:
#testing goal 2.4
goal[10]
#goal_id[10]

u'2.4 By 2030, ensure sustainable food production systems and implement resilient agricultural practices that increase productivity and production, that help maintain ecosystems, that strengthen capacity for adaptation to climate change, extreme weather, drought, flooding and other disasters and that progressively improve land and soil quality\xa0'